In [4]:
import manim as manim
import numpy as np
import copy
import colormap
import palettable

In [5]:
%manim --help

Usage: manim [OPTIONS] COMMAND [ARGS]...

  Animation engine for explanatory math videos

Options:
  --version  Show version and exit.
  --help     Show this message and exit.

Commands:
  render*  Render SCENE(S) from the input FILE.
  cfg      Manages Manim configuration files.
  plugins  Manages Manim plugins.

  Made with <3 by Manim Community developers.


In [8]:
colors = ['#caf0f8', '#90e0ef', '#00b4d8', '#0096c7']
# Gradient generated at: https://coolors.co/gradient-palette/83a566-628444?number=7
result_colors = ['#caffbf', '#BEF0B0', '#B2E1A1', '#A7D293', '#9BC384', '#8FB475', '#83A566', '#7EA060', '#789A5B', '#739555', '#6D8F4F', '#688A4A'] 
#result_colors = YlGn_9.hex_colors #Deep_20.hex_colors
result_colors = [colormap.rgb2hex(*palettable.cartocolors.sequential.Emrld_7.mpl_colormap(n)) for n in np.linspace(0.0, 0.5, 10)]
result_color = result_colors[0]

use_gradient = True
mixes = np.array([[2, 4, 1], [1, 1, 1], [0, 3, 2], [0, 1, 0], [4, 1, 0], [3, 0, 0], [2, 2, 2], [1, 1, 3], [1, 0, 2]])
X_in =  np.array([[1, 0, 1, 3], [1, 4, 0, 0], [0, 0, 2, 0]])
bg_color = '#f0f0f0'
font_color = '#0a0a0a'
line_color = '#7b7b7b'
scale = 0.5
target_point = np.array([1, -5, 0])*scale
source_point = np.array([0, 2, 0])
num_input_boxes = 4
num_inputs = 3
mix = [2, 4, 1]
publish = True
manim.config["background_color"] = bg_color
if publish:
    manim.config.quality = 'production_quality'
    manim.config.pixel_width = 1280
    manim.config.pixel_height = 720
    manim.config.frame_rate = 60
else:
    manim.config.quality = 'low_quality'
    manim.config.pixel_width = 640
    manim.config.pixel_height = 360
    manim.config.frame_rate = 60


In [16]:
%%manim -qh -r 1280,720 -o combining-columns.mp4 CombiningColumns

#%%manim -qh -r 1280,720 -o linear-column-equation.png CombiningColumns

manim.config.quality = 'production_quality'
manim.config.pixel_width = 1280
manim.config.pixel_height = 720
manim.config.frame_rate = 60

scale = 0.5

   
class CombiningColumns(manim.MovingCameraScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.vecs = []
        self.res = None
        self.factors = []
        self.equals = None
        self.other_text = [] 
        
    def construct(self):
        self.construct_to_matrix_notation()
        #self.construct_adding_examples()
        
    def construct_to_matrix_notation(self):
        self.camera.frame_center = manim.ORIGIN - np.array([2.1, 0, 0])        
        eq1 = self.eq1()
        eq2_target = self.eq2()
        animations = self.anims(eq1, eq2_target)
        self.add(eq1)
        #return # return here for img
        self.wait(1)
        self.play(*animations)
        self.add(eq2_target.obj)
        self.remove(*self.other_text)
        self.wait(1)
        
    def construct_adding_examples(self):
        self.camera.frame_center = manim.ORIGIN - np.array([-3.5, 0, 0]) 
        self.camera.frame.scale(1.3)
        self.eq1()
        eq2 = self.eq2()
        self.add(eq2.obj)
        self.wait(1)
        def create_num_col(v):
            return manim.Group(*(manim.Integer(i, color=font_color) for i in v)).arrange(direction=-manim.Y_AXIS)

        fs = [eq2.m['F']]
        ys = [eq2.m['Y']]
        # Without partials, it's easy to accidentally not capture a copy of a variable in a lambda.
        import functools
        def update_y(prev_y, y):
            y.next_to(prev_y)
            
        ys[0].add_updater(lambda o : o.next_to(eq2.m['lb3']))        
        for i in range(1, len(mixes)):          
            ys.append(copy.deepcopy(ys[i-1]).next_to(ys[i-1]).set_fill(result_colors[i]))
            fs.append(create_num_col(mixes[i]).next_to(fs[i-1]))
            ys[i].add_updater(functools.partial(update_y, ys[i-1]))
        eq2.m['='].add_updater(lambda o : o.next_to(eq2.m['rb2']))
        eq2.m['lb3'].add_updater(lambda o : o.next_to(eq2.m['=']))
        current = 0
        #eq2.m['rb3'].add_updater(lambda o : o.next_to(ys[current]))
        for i in range(1, len(mixes)):
            anims = []
            # I can't get next_to to work for rb3. So use hacky shift instead.
            #self.play(eq2.m['rb3'].animate.next_to(ys[i]))            
            self.play(eq2.m['rb2'].animate.next_to(fs[i]),
                      eq2.m['rb3'].animate.shift([2.4*scale, 0, 0]))
            current = i
            anims.append(manim.FadeIn(fs[i]))
            anims.append(manim.FadeIn(ys[i]))
            self.play(*anims)
            self.wait(0.5)
        self.wait(1)
        
        # Part 2.
        to_nums_anim = []
        to_blocks_anim = []
        y = mixes @ X_in
        block_num_pairs = list(zip([eq2.m['X0'], eq2.m['X1'], eq2.m['X2']], X_in)) + list(zip(ys, y))
        for c,nums in block_num_pairs:
            for block, n in zip(reversed(c), nums):
                manim_int = manim.Integer(n, color=font_color).move_to(block)
                to_nums_anim.append(manim.FadeIn(manim_int))
                to_nums_anim.append(manim.FadeOut(block))
                to_blocks_anim.append(manim.FadeOut(manim_int))
                to_blocks_anim.append(manim.FadeIn(block))
        self.play(*to_nums_anim)      
        self.wait(1.3)
        self.play(*to_blocks_anim)
        self.wait(1.3)
        self.play(*to_nums_anim)
        self.wait(1.3)
        self.play(*to_blocks_anim)
        self.wait(1)
    
    def eq2(self):
        lb = manim.MathTex(r'\big[', color=font_color).scale(2).stretch_to_fit_height(self.vecs[0].height + 0.2)
        rb = manim.MathTex(r'\big]', color=font_color).scale(2).stretch_to_fit_height(lb.height)
        eq2_map = TargetObject(OrderedDict({
                    #manim.IntegerMatrix([[mix[0]], [mix[1]], [mix[2]]]).set_color(font_color),
                    'lb1' : lb.copy(),
                    'X0'  : copy.deepcopy(self.vecs[0]),
                    'X1'  : self.vecs[1].copy(),
                    'X2'  : self.vecs[2].copy(),
                    'rb1' : rb.copy(),
                    'lb2' : lb.copy(),
                    'F'   : manim.Group(*(manim.Integer(i, color=font_color) for i in mix)).arrange(direction=-manim.Y_AXIS),
                    'rb2' : rb.copy(),             
                    '='   : manim.MathTex('=', color=font_color),
                    'lb3' : lb.copy(),
                    'Y'   : self.res.copy(),
                    'rb3' : rb.copy()}),
                    draw_exclude={'F', 'X0', 'X1', 'X2', 'Y', '='})
        eq2_map.obj.arrange(direction=manim.X_AXIS)
        # Maintain position of equals sign.
        equality_sign_disp = eq2_map.m['='].get_center() - self.equals.get_center()
        eq2_map.obj.shift(-equality_sign_disp)
        return eq2_map
        
        
    def anims(self, eq1, eq2_map):
        replacements = (
            (self.vecs[0],    eq2_map.m['X0']),
            (self.vecs[1],    eq2_map.m['X1']),      
            (self.vecs[2],    eq2_map.m['X2']), 
            (self.res,        eq2_map.m['Y']),
            (self.factors[0], eq2_map.m['F'][0]),
            (self.factors[1], eq2_map.m['F'][1]),
            (self.factors[2], eq2_map.m['F'][2]),
            (self.equals,     eq2_map.m['='])
        )
        animations = [*(manim.ReplacementTransform(a, b) for a,b in replacements),
                      *(manim.FadeIn(o) for o in eq2_map.elements_to_draw()),
                     #manim.FadeIn(eq2_map.obj),
                      *(manim.FadeOut(o) for o in self.other_text)]
        return animations
       
    
    def eq1(self):
        eq = manim.Mobject()
        spacing = 4.3 * scale
        offsets = np.array([1, 0.3, 0.3, 0.5, 1.15]) * scale
        
        for i in range(len(mix)):
            vec = create_2d_vec(num_input_boxes, manim.Y_AXIS, colors[i])
            vec_pos = spacing * (-len(mix) + i) + offsets[0]
            vec.move_to([vec_pos, 0, 0])
            mult = manim.Tex('$\\times$', color=font_color)
            mult.next_to(vec, buff=offsets[1])
            factor = manim.Tex(f'${mix[i]}$', color=font_color)
            factor.next_to(mult, buff=offsets[2])
            if(i < len(mix) - 1):
                plus = manim.Tex('$+$', color=font_color)
                plus.next_to(factor, buff=offsets[3])
            self.vecs.append(vec)
            self.factors.append(factor)
            self.other_text.extend([mult, plus])
            eq.add(vec, factor, mult, plus)
        self.equals = manim.Tex('=', color=font_color)
        self.equals.move_to([0, 0, 0])
        self.res = create_2d_vec(num_input_boxes, manim.Y_AXIS, result_color)
        self.res.next_to(self.equals, buff=offsets[4])
        eq.add(self.equals, self.res)
        eq.add(self.equals)
        return eq

ValueError: latex error converting to dvi. See log output above or the log file: media/Tex/1413b97f288b63f9.log

In [10]:
%%manim -qh -r 1280,720 -o combining-columns-extension.mp4 TransitionToNumbers

class TransitionToNumbers(CombiningColumns):
    def construct(self):
        super().construct_adding_examples()

NameError: name 'create_2d_vec' is not defined

In [17]:
%%manim -qh -r 1280,720 -o rows-combine-rows.mp4 RowsCombineRows


def create_bracket(length=4.0, side_length=0.14, open_dir=manim.RIGHT):
    obj = manim.VMobject(stroke_color=font_color)
    obj.set_points_as_corners([np.array(p) for p in ((0, 0, 0), (side_length, 0, 0), (side_length, length, 0), (0,length, 0))])
    if np.array_equal(open_dir, manim.LEFT):
        pass
    elif np.array_equal(open_dir, manim.RIGHT):
        obj.flip(manim.Y_AXIS)
    elif np.array_equal(open_dir, manim.UP):
        obj.rotate_about_origin(manim.DEGREES*90)
    elif np.array_equal(open_dir == manim.DOWN):
        obj.flip(manim.Y_AXIS)
        obj.rotate_about_origin(manim.DEGREES*90)
    else:
        raise Exception(f'Unexpected direction: {open_dir}')
    return obj


# Can't seem to get the svg version to work
def create_brace_svg(length=1.0, side_length=0.2, open_dir=manim.RIGHT, stroke_width=0):
    path_string_template = f'M0 0L{side_length} 0L{side_length} {length}L{0} {side_length}z'
    obj = manim.SVGPathMobject(path_string=path_string_template, 
                               stroke_width=stroke_width,
                               stroke_color=font_color,
                               sharpness=2).set_color(font_color)
    if np.array_equal(open_dir, manim.LEFT):
        pass
    elif np.array_equal(open_dir, manim.RIGHT):
        obj.flip(manim.X_AXIS)
    elif np.array_equal(open_dir, manim.UP):
        obj.rotate_about_origin(manim.DEGREES*90)
    elif np.array_equal(open_dir == manim.DOWN):
        obj.flip(manim.X_AXIS)
        obj.rotate_about_origin(manim.DEGREES*90)
    else:
        raise Exception(f'Unexpected direction: {open_dir}')
    return obj




class RowsCombineRows(manim.MovingCameraScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.vecs = []
        self.res = None
        self.factors = []
        self.equals = None
        self.other_text = [] 
        
    def construct(self):
        self.camera.frame_center = manim.ORIGIN + np.array([0.7, -2.5, 0])
        self.camera.frame.scale(1.2)        
        self.eq_animated()
        return

        fs = [eq2.m['F']]
        ys = [eq2.m['Y']]
        # Without partials, it's easy to accidentally not capture a copy of a variable in a lambda.
        import functools
        def update_y(prev_y, y):
            y.next_to(prev_y)
            
        ys[0].add_updater(lambda o : o.next_to(eq2.m['lb3']))        
        for i in range(1, len(mixes)):
            ys.append(copy.deepcopy(ys[i-1]).next_to(ys[i-1]))
            fs.append(create_num_col(mixes[i]).next_to(fs[i-1]))
            ys[i].add_updater(functools.partial(update_y, ys[i-1]))
        eq2.m['='].add_updater(lambda o : o.next_to(eq2.m['rb2']))
        eq2.m['lb3'].add_updater(lambda o : o.next_to(eq2.m['=']))
        current = 0
        #eq2.m['rb3'].add_updater(lambda o : o.next_to(ys[current]))
        for i in range(1, len(mixes)):
            anims = []
            # I can't get next_to to work for rb3. So use hacky shift instead.
            #self.play(eq2.m['rb3'].animate.next_to(ys[i]))            
            self.play(eq2.m['rb2'].animate.next_to(fs[i]),
                      eq2.m['rb3'].animate.shift([2.4*scale, 0, 0]))
            current = i
            anims.append(manim.FadeIn(fs[i]))
            anims.append(manim.FadeIn(ys[i]))
            self.play(*anims)
            self.wait(0.5)
        self.wait(1)
        
        # Part 2.
        to_nums_anim = []
        to_blocks_anim = []
        y = mixes @ X_in
        block_num_pairs = list(zip([eq2.m['X0'], eq2.m['X1'], eq2.m['X2']], X_in)) + list(zip(ys, y))
        for c,nums in block_num_pairs:
            for block, n in zip(reversed(c), nums):
                manim_int = manim.Integer(n, color=font_color).move_to(block)
                to_nums_anim.append(manim.FadeIn(manim_int))
                to_nums_anim.append(manim.FadeOut(block))
                to_blocks_anim.append(manim.FadeOut(manim_int))
                to_blocks_anim.append(manim.FadeIn(block))
        self.play(*to_nums_anim)      
        self.wait(1.3)
        self.play(*to_blocks_anim)
        self.wait(1.3)
        self.play(*to_nums_anim)
        self.wait(1.3)
        self.play(*to_blocks_anim)
        self.wait(1)
   
    def create_num_row(self, v):
        return manim.Group(*(manim.Integer(i, color=font_color) for i in v)).arrange(direction=manim.X_AXIS, buff= 0.5)

    def eq_animated(self):
        xs = manim.Group()
        x_vecs = [create_2d_vec(num_input_boxes, manim.X_AXIS, colors[i]) for i in range(len(mix))] 
        xs.add(*reversed(x_vecs))
        xs.arrange(direction=manim.Y_AXIS)        
        xs.move_to(manim.ORIGIN)
        lb_x = manim.MathTex(r'\big[', color=font_color).scale(3).stretch_to_fit_height(xs.height + 0.4).next_to(xs, manim.LEFT)
        rb_x = manim.MathTex(r'\big]', color=font_color).scale(3).stretch_to_fit_height(lb_x.height) .next_to(xs, manim.RIGHT)
        lb_x = create_bracket(xs.height + 0.4, open_dir=manim.RIGHT).next_to(xs, manim.LEFT)
        rb_x = create_bracket(xs.height + 0.4, open_dir=manim.LEFT).next_to(xs, manim.RIGHT)
        equality = manim.MathTex('=', color=font_color).next_to(rb_x, manim.RIGHT)
        self.add(xs, lb_x, rb_x, equality)
        ys = [create_2d_vec(num_input_boxes, manim.X_AXIS, result_color)]
        fs = [self.create_num_row(mixes[0])]
        f_buff = np.array([-2, 0, 0])
        y_buff = np.array([3, 0, 0])
        fs[0].move_to(x_vecs[0].get_critical_point([-1, 0, 0]) + f_buff)
        ys[0].move_to(x_vecs[0].get_critical_point([1, 0, 0]) + y_buff)        
        self.add(ys[0], fs[0])

        def bracket_height():
            return ys[0].get_critical_point([0, 1, 0])[1] - ys[-1].get_critical_point([0, -1, 0])[1] + bracket_margin
        bracket_margin = 0.4
        #lb_f = manim.MathTex(r'\big[', color=font_color).scale(2).stretch_to_fit_height(ys[0].height + bracket_margin).next_to(fs[0], manim.LEFT)
        lb_f = create_bracket(bracket_height()).next_to(fs[0], manim.LEFT)
        rb_f = create_bracket(bracket_height(), open_dir=manim.LEFT).next_to(fs[0], manim.RIGHT)
        lb_y = create_bracket(bracket_height()).next_to(ys[0], manim.LEFT)
        rb_y = create_bracket(bracket_height(), open_dir=manim.LEFT).next_to(ys[0], manim.RIGHT)
        self.add(lb_f, rb_f, lb_y, rb_y)
        
        cur = 0
        bracket_incr = 0.7
        for i in range(1, len(mixes)):
            cur += 1
            self.wait(1)
            y_pos = ys[-1].get_critical_point(np.array([0, -1, 0]))
            f_pos = fs[-1].get_critical_point(np.array([0, -1, 0]))
            buff = np.array([0, -0.5, 0])
            new_y_pos = y_pos + buff
            res_color = result_colors[i] if use_gradient else result_color
            ys.append(create_2d_vec(num_input_boxes, manim.X_AXIS, res_color).move_to(new_y_pos))
            new_f_pos = np.array([f_pos[0], new_y_pos[1], 0])
            fs.append(self.create_num_row(mixes[i]).move_to(new_f_pos))
            self.play(*[
                lb_f.animate.become(create_bracket(bracket_height(), open_dir=manim.RIGHT).align_to(lb_f, direction=[-1,1, 0])),
                rb_f.animate.become(create_bracket(bracket_height(), open_dir=manim.LEFT ).align_to(rb_f, direction=[-1,1, 0])),
                lb_y.animate.become(create_bracket(bracket_height(), open_dir=manim.RIGHT).align_to(lb_y, direction=[-1,1, 0])),
                rb_y.animate.become(create_bracket(bracket_height(), open_dir=manim.LEFT ).align_to(rb_y, direction=[-1,1, 0])),
            
            ])
            self.play(*[manim.FadeIn(ys[-1]), manim.FadeIn(fs[-1])])

class unused:
    def eq(self):
        xs = manim.Group()
        ys = manim.Group()  
        fs = manim.Group()        
        xs.add(*[create_2d_vec(num_input_boxes, manim.X_AXIS, colors[i]) for i in range(len(mix))])
        ys.add(*[create_2d_vec(num_input_boxes, manim.X_AXIS, result_colors[i]) for i in range(len(mixes))])
        fs.add(*[self.create_num_row(r) for r in mixes])
        xs.arrange(direction=manim.Y_AXIS)        
        ys.arrange(direction=manim.Y_AXIS)          
        fs.arrange(direction=manim.Y_AXIS)
        lb_tall = manim.MathTex(r'\big[', color=font_color).scale(2).stretch_to_fit_height(fs.height + 0.4)
        rb_tall = manim.MathTex(r'\big]', color=font_color).scale(2).stretch_to_fit_height(lb_tall.height)        
        lb_short = manim.MathTex(r'\big[', color=font_color).scale(2).stretch_to_fit_height(xs.height + 0.4)
        rb_short = manim.MathTex(r'\big]', color=font_color).scale(2).stretch_to_fit_height(lb_short.height)         
        eq = TargetObject(OrderedDict({
                    #manim.IntegerMatrix([[mix[0]], [mix[1]], [mix[2]]]).set_color(font_color),
                    'lb1' : lb_tall.copy(),
                    'F'   : fs,
                    'rb1' : rb_tall.copy(),
                    'lb2' : lb_short.copy(),
                    'X'   : xs,            
                    'rb2' : rb_short.copy(),             
                    '='   : manim.MathTex('=', color=font_color),
                    'lb3' : lb_tall.copy(),
                    'Y'   : ys,
                    'rb3' : rb_tall.copy()}))
        # Maintain position of equals sign.
        def stretch_to_y_height(o):
            pass#o.stretch_to_fit_height(eq.m['Y'].height + 0.4)
        
        def stretch_to_f_height(o):
            pass#o.stretch_to_fit_height(eq.m['F'].height + 0.4)
            
        eq.m['lb1'].add_updater(stretch_to_f_height)
        eq.m['rb1'].add_updater(stretch_to_f_height)
        eq.m['lb3'].add_updater(stretch_to_y_height)
        eq.m['rb3'].add_updater(stretch_to_y_height)
        eq.obj.arrange(direction=manim.X_AXIS)
        anims = []
        return eq
    
    def eq1(self):
        eq = manim.Mobject()
        spacing = 4.3 * scale
        offsets = np.array([1, 0.3, 0.3, 0.5, 1.15]) * scale
        
        for i in range(len(mix)):
            vec = create_2d_vec(num_input_boxes, manim.Y_AXIS, colors[i])
            vec_pos = spacing * (-len(mix) + i) + offsets[0]
            vec.move_to([vec_pos, 0, 0])
            mult = manim.Tex('$\\times$', color=font_color)
            mult.next_to(vec, buff=offsets[1])
            factor = manim.Tex(f'${mix[i]}$', color=font_color)
            factor.next_to(mult, buff=offsets[2])
            if(i < len(mix) - 1):
                plus = manim.Tex('$+$', color=font_color)
                plus.next_to(factor, buff=offsets[3])
            self.vecs.append(vec)
            self.factors.append(factor)
            self.other_text.extend([mult, plus])
            eq.add(vec, factor, mult, plus)
        self.equals = manim.Tex('=', color=font_color)
        self.equals.move_to([0, 0, 0])
        self.res = create_2d_vec(num_input_boxes, manim.Y_AXIS, result_color)
        self.res.next_to(self.equals, buff=offsets[4])
        eq.add(self.equals, self.res)
        eq.add(self.equals)
        return eq        

ValueError: latex error converting to dvi. See log output above or the log file: media/Tex/3b9613821610fd57.log

In [12]:
%%manim -qh -r 1280,720 -o many-times.mp4 CRecipe

def create_3d_vec(segments, color=None, opacity=1.0):
    box_size = 1*scale
    vector_ob = manim.VGroup()
    prisms = []
    for s in range(segments):
        p = manim.Prism([box_size, box_size, box_size])
        p.set_fill(color, opacity=opacity)
        p.align_on_border(manim.IN)
        p.set_stroke(color='#000000', width=1.0)
        prisms.append(p)
    vector_ob.add(*prisms)
    vector_ob.arrange(direction=-manim.Y_AXIS, buff=0)
    return vector_ob


def surrounding_box(mobject, buff=0):
    prism = manim.Prism([mobject.width+buff, mobject.height+buff, mobject.depth+buff])
    prism.move_to(mobject.get_critical_point(direction=manim.ORIGIN))
    return prism


def apply_scaffold_style(mobject):
    mobject.set_fill(opacity=0.1, color=bg_color)
    mobject.set_stroke(opacity=0.3)
    return mobject


def create_vec_list(num_vecs, segments, pos):
    vec_list = manim.VMobject()
    for i in range(num_vecs):
        v = create_3d_vec(segments, colors[i])
        vec_list.add(v)
        vec_list.arrange(direction=manim.RIGHT) 
        vec_list.move_to(pos)
    return vec_list

    
def create_mix_nums(vec_list, mix):
    mix_decimals = manim.VGroup()
    offset = np.array([0, -1.2, -0.5]) * scale
    for i in range(len(mix)):
        pos = vec_list[i].get_edge_center(manim.DOWN) + offset
        num = manim.DecimalNumber(mix[i], num_decimal_places=0, color=font_color)
        num.move_to(pos)
        mix_decimals.add(num)
    return mix_decimals


def create_mix_result(vec_list, mix, target_point, color=result_color):
    dup_vecs = manim.VGroup()
    shift_dist = vec_list[0].depth + 0
    offset = 2 * scale
    dup_anims = []
    for i in range(len(vec_list)):
        dup_i = [copy.deepcopy(vec_list[i]) for j in range(mix[i])]
        for j in range(0, len(dup_i)):
            dist = j * shift_dist + offset
            dup_anims.append(dup_i[j].animate.shift(dist * manim.OUT))
        dup_vecs.add(*dup_i)
    def animation(scene):
        scene.add(dup_vecs)
        scene.play(*dup_anims)
        dup_vecs.set_fill(color)
        animations = [dv.animate.move_to(target_point) for dv in dup_vecs]
        def add_surrounding_box():
            bb = surrounding_box(dup_vecs, buff=0.2)   
            bb.set_color(color)
            bb.set_opacity(0.2)
            scene.add(bb)
            scene.wait(0.2)
            animations.extend([bb.animate.move_to(target_point), manim.ReplacementTransform(bb, replacement_vec)])
        # this doesn't work well enough to use.
        #add_surrounding_box()
        scene.play(*animations)
        scene.remove(dup_vecs)
    replacement_vec = create_3d_vec(num_input_boxes, color=color)
    replacement_vec.move_to(target_point)
    return replacement_vec, animation


def create_mix_lines(vec_list, target_point):
    lines = []
    for i in range(len(vec_list)):
        start_point = vec_list[i].get_edge_center(manim.DOWN)
        # Should we move the start point to be at the bottom of the box?
        end_point = target_point + np.array([0, 2.05, -0.5])*scale
        l = manim.DashedLine(start_point, end_point, dash_length=0.02, 
                             positive_space_ratio=0.2, color=line_color)
        # Doesn't seem to have an effect.
        #l.set_opacity(0.7)
        lines.append(l)
    g1 = manim.Group(*lines)
    return g1

def create_2d_vec(segments, direction=manim.X_AXIS, color=None, opacity=1.0):
    box_size = 1*scale
    vector_ob = manim.VMobject()
    rects = []
    for s in range(segments):
        r = manim.Square(side_length=box_size, color=color, fill_opacity=opacity)
        #r.align_on_border(manim.X_AXIS)
        r.set_stroke(color='#000000', width=1.0)
        rects.append(r)
    vector_ob.add(*rects)
    vector_ob.arrange(direction=direction, buff=0)
    return vector_ob
    
from collections import OrderedDict

class TargetObject():
    def __init__(self, obj_map : OrderedDict, draw_exclude=None, **kwargs):
        self.obj = manim.Mobject(**kwargs)
        self.obj.add(*obj_map.values())
        self.m = obj_map
        self.draw_exclude = set() if not draw_exclude else set(draw_exclude)
        
    def elements_to_draw(self):
        elems = [v for k,v in self.m.items() if k not in self.draw_exclude]
        return elems


class ARecipe(manim.ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=55*manim.DEGREES, theta=-115*manim.DEGREES, distance=400.0)
        #self.camera.background_color = '#f0f0f0'
        # Moving camera doesn't seem to work. A black dot is created.
        #self.move_camera(frame_center=target_point)
        vec_list = create_vec_list(num_inputs, num_input_boxes, pos=source_point)
        self.add(vec_list)        
        self.add(apply_scaffold_style(create_vec_list(1, num_input_boxes, pos=target_point)))
        self.add(create_mix_lines(vec_list, np.array(target_point)))
        self.add(*create_mix_nums(vec_list, mix))
        res_vec, animation = create_mix_result(vec_list, mix, target_point)
        animation(self)
        self.add(res_vec)
        self.wait()

    
class BRecipe(ARecipe):
    def construct(self):
        self.set_camera_orientation(phi=55*manim.DEGREES, theta=-115*manim.DEGREES, distance=400.0)
        vec_list = create_vec_list(num_inputs, num_input_boxes, pos=source_point)
        self.add(vec_list)        
        self.add(create_mix_lines(vec_list, np.array(target_point)))
        self.add(*create_mix_nums(vec_list, mix))
        res_vec, animation = create_mix_result(vec_list, mix, target_point)
        self.add(res_vec)
        num_extensions = 4
        def extend_vec(v, direction):
            fixed_ref = np.array([0, 0, 0]) - direction
            fixed_point = v.get_critical_point(fixed_ref)
            to_append = copy.deepcopy(v[-1])
            v.add(to_append)
            v.arrange(direction=manim.Y_AXIS, buff=0)
            v.move_to(fixed_point, aligned_edge=fixed_ref) 
            return to_append

        def extend_vecs():
            for i in range(num_extensions):
                self.wait(0.5)
                added = []
                for v in vec_list:
                    added.append(extend_vec(v, direction=np.array([0, 1, 0])))
                added.append(extend_vec(res_vec, direction=np.array([0, -1, 0])))
                # Some horrible z-order issues make this annimation not work well.
                #self.play(*[manim.FadeIn(b) for b in added])
        def shrink_vecs():
            for i in range(num_extensions):
                self.wait(0.1)
                for v in vec_list:
                    v.remove(v[-1])
                res_vec.remove(res_vec[0])
        extend_vecs()
        self.wait(1)
        shrink_vecs()
        self.wait()   
        
        
class CRecipe(manim.ThreeDScene):
    def construct(self):
        offset = np.array([-3, 0, 0])
        self.set_camera_orientation(phi=55*manim.DEGREES, theta=-115*manim.DEGREES, distance=400.0)
        source_vecs = create_vec_list(num_inputs, num_input_boxes, pos=source_point + offset)
        self.add(source_vecs)            
        placement_point = np.array([1, -5, 0])*scale + offset
        mix_nums = None
        mix_lines = None
        for i, m in enumerate(mixes):
            # 1. Lines and numbers
            if mix_nums:
                mix_nums_next = create_mix_nums(source_vecs, m)
                mix_lines_next = create_mix_lines(source_vecs, np.array(placement_point))
                self.play(manim.ReplacementTransform(mix_nums, mix_nums_next), 
                          manim.ReplacementTransform(mix_lines, mix_lines_next), 
                          manim.FadeIn(apply_scaffold_style(create_vec_list(1, num_input_boxes, pos=placement_point))),run_time=0.4)
                mix_nums = mix_nums_next
                mix_lines = mix_lines_next
            else:
                assert mix_lines == None
                mix_nums = create_mix_nums(source_vecs, m)
                mix_lines = create_mix_lines(source_vecs, np.array(placement_point))
                self.add(*mix_nums, *mix_lines)
                self.add(apply_scaffold_style(create_vec_list(1, num_input_boxes, pos=placement_point)))
                
            # 2. Mix animation and result
            # The gradient just seems a bit distracting.
            color = result_colors[i] if use_gradient else result_color
            res_vec, animation = create_mix_result(source_vecs, m, placement_point, color=color)
            animation(self)
            self.add(res_vec)
            self.wait(0.5)
            placement_point = placement_point + np.array([1.5, 0, 0]) * scale

ValueError: latex error converting to dvi. See log output above or the log file: media/Tex/db6edc3108d2d38a.log

In [ ]:
%%manim -qh -r 1280,720 -o one-combination.mp4 ARecipe
pass

In [ ]:
%%manim -qh -r 1280,720 -o resize-objects.mp4 BRecipe
pass

In [ ]:
%%manim -ql My2DScene

import functools as ft
import copy
colors = ['#caf0f8', '#90e0ef', '#00b4d8', '#0096c7']

class MyMatrix(manim.DecimalMatrix):
    pass

M = np.array([[2, 0.5, 1],[0.5, 1, 1]])
M0 = np.zeros(M.shape)
M = M0
W = np.array([[3, 1, 4,1],[0, 2, 1, 0.5],[0.5, 0, 2, 0]])
add1 = np.zeros(M.shape)
add1[0, 1] = 1.0
Z = M0 @ W

class My2DScene(manim.Scene):
    def construct(self):
        M_matrix = MyMatrix(M)
        M0_matrix = MyMatrix(M0)
        Z_matrix = MyMatrix(Z)
        W_matrix = MyMatrix(W)
        arrow = manim.Arrow(start=manim.LEFT, end=manim.RIGHT, color='#505050')
        matrix_eq = manim.VMobject()
        #matrix_eq.add(M0_matrix, W_matrix, arrow, Z_matrix)
        #matrix_eq.arrange()
        #self.add(matrix_eq)
        
        #matrix_mult_ob.arrange()
        #W_matrix.next_to(M0_matrix, manim.RIGHT)        
        #arrow.next_to(W_matrix, manim.RIGHT)
        #Z_matrix.next_to(arrow, manim.RIGHT)
        #self.add(M0_matrix, W_matrix, arrow, Z_matrix)
        #dup_row1 = copy.deepcopy(W_matrix.get_rows()[0])
        #target_x = W_matrix.get_center()[1]
        #y_offset = 2
        #target_y = W_matrix.get_critical_point(manim.DOWN)[0] - 2
        #animate_row_mult(self, dup_row1, 2.0, np.array([target_x, target_y, 0]))
        #matrix_mult_ob.add(M_matrix, W_matrix)
        #matrix_mult_ob.arrange()
        #self.add(matrix_mult_ob)
        
        def plus(to_add, m, dt):
            if m.mob_matrix.shape != to_add.shape:
                raise Exception("shapes must match")
            rate = 0.2
            factor = min(1.0, dt*rate)
            for r in range(m.mob_matrix.shape[0]):
                for c in range(m.mob_matrix.shape[1]):
                    m.mob_matrix[r,c].set_value(m.mob_matrix[r,c].get_value() + 
                                                to_add[r,c]*factor)
                    
                    
        def plus_complete(eq, dt):
            rate = 0.4
            factor = min(1.0, dt*rate)
            print(dt)
            global M
            M = factor*add1 + M
        
        def get_M():
            return MyMatrix(M).next_to(W_matrix, manim.LEFT)
            
        def get_mult_result():
            y = M @ W
            y_matrix = MyMatrix(y).next_to(arrow, manim.RIGHT)
            return y_matrix
        
        matrix_eq.add(manim.always_redraw(get_M), 
                      W_matrix, arrow, manim.always_redraw(get_mult_result).next_to(arrow))
        matrix_eq.add_updater(plus_complete)
        matrix_eq.arrange()
        self.add(matrix_eq)
        #M_matrix.add_updater(ft.partial(plus, negM))
        #matrix_eq.add_updater(lambda meq : meq.arrange())
        self.wait(3)
        
                    
            
def animate_row_mult(scene, row, multiplier, target_pos, alignment=manim.ORIGIN):
    init_values = [d.get_value() for d in row]
    get_dist = lambda : np.linalg.norm(target_pos - row.get_critical_point(alignment))
    init_dist = get_dist()

    def updater(row):
        for i, d in enumerate(row):
            current_dist = get_dist()
            progress = round(1 - current_dist/init_dist, 10)
            temp_mult = 1 * (1 - progress) + multiplier * progress
            d.set_value(init_values[i] * temp_mult)
    #    decimal.add_updater(ft.partial(updater, init_val=init_values[i]))
    #row.add_updater(updater)
    #scene.play(row.animate.move_to(target_pos, alignment), run_time=3)
    scene.play(row.animate.shift(target_pos - row.get_critical_point(alignment)))
    scene.wait(1)

In [13]:
%%manim -ql MyScene

colors = ['#caf0f8', '#90e0ef', '#00b4d8', '#0096c7']

M = np.array([[2, 0.5, 1],[0.5, 1, 1]])
W = np.array([[2, 1, 4,1],[0, 2, 1, 0.5],[0.5, 0, 2, 0]])

class MyScene(manim.ThreeDScene):     
    def construct(self):
        #self.move_camera(distance=90*scale)
        #self.set_camera_orientation(phi=50*manim.DEGREES, theta=-60*manim.DEGREES)
        self.set_camera_orientation(phi=0, theta=-90*manim.DEGREES)
        axes = manim.ThreeDAxes()
        self.add(axes)
        M_prisms = self.matrix_as_prisms(M)
        W_prisims = self.matrix_as_prisms(W)
        M_matrix = manim.Matrix(M)
        W_matrix = manim.Matrix(W)
        #M_matrix.move_to(manim.ORIGIN, aligned_edge=[-1, 1, -1])
        #W_matrix.next_to(M_matrix, manim.RIGHT)
        matrix_mult_ob = manim.VMobject()
        matrix_mult_ob.add(M_matrix, W_matrix)
        matrix_mult_ob.arrange()
        matrix_mult_ob.move_to(manim.ORIGIN, aligned_edge=[-1, 1, -1])
        #self.add(M_matrix, W_matrix)
        self.add(matrix_mult_ob)
        self.move_camera(frame_center=np.array([10,50,0]))#matrix_mult_ob.get_center())
        #self.move_camera(phi=45*manim.DEGREES)
        #self.begin_ambient_camera_rotation(rate=0.25)
        self.wait(3)
        #self.stop_ambient_camera_rotation()
        
    def matrix_as_prisms(self, matrix, max_opacity=1.0, min_opacity=1.0):
        matrix_ob = manim.VMobject()
        opacity_step = (max_opacity - min_opacity) / matrix.shape[0]
        prisms = []
        for r in range(matrix.shape[0]):
            #for c in range(matrix.shape[1]):
             #   p = manim.Prism([width, depth, matrix[r, c]*scale])
             #   p.move_to(c*scale * manim.RIGHT + (r + r * row_spacing) * scale * manim.DOWN)#-0.5*matrix[r,c]*IN)
             #   p.align_on_border(manim.IN)
             #   p.set_fill(colors[r], opacity=max_opacity - r*opacity_step)
             #   p.set_stroke(color='#00000', width=1.0)
             #   matrix_ob.add(p)
            vec = self.vector_as_prisms(matrix[r], colors[r], opacity=max_opacity - r*opacity_step)
            matrix_ob.add(vec)
        matrix_ob.arrange(direction=np.array([0,-1,0]), center=False, coor_mask=np.array([0,1,0]))
        #matrix_ob.space_out_submobjects(factor=0.1)
        return matrix_ob
    
    def vector_as_prisms(self, vector, color=None, opacity=1.0):
        width = depth = 1
        vector_ob = manim.VMobject()
        prisms = []
        i = 0
        for val in vector:
            p = manim.Prism([width, depth, val])
            p.set_fill(color, opacity=opacity)
            p.align_on_border(manim.IN)
            #p.set_stroke(color='#00000', width=1.0)
            #bottom_left_corner = p.get_critical_point([-1,-1,-1])
            #p.shift(-val*scale*0.5*np.array([0,0,1]))
            #p.shift(np.array([2,0,0])*i)
            #i+=1
            prisms.append(p)
        vector_ob.add(*prisms)
        #vector_ob.to_edge(edge=manim.IN)
        vector_ob.arrange(direction=manim.X_AXIS, center=False, coor_mask=np.array([1,0,0]))
        return vector_ob
            
                
                


ValueError: latex error converting to dvi. See log output above or the log file: media/Tex/33bdc51ecdcab2ca.log

In [14]:
# Graveyard



class CombiningColumns3(manim.Scene):
    def construct(self): 
        vecs = [create_2d_vec(num_input_boxes, manim.Y_AXIS, colors[i]) for i in range(len(mix))]
        vec_identifiers = ['X0', 'X1', 'X2', 'Y']
        factor_identifiers = ['f0', 'f1', 'f2']
        all_identifiers = vec_identifiers + factor_identifiers + ['=']
        
        def repl(eq1, tex, obj):
            idx = eq1.index_of_part_by_tex(tex)
            repl = eq1[idx].become(obj.move_to(eq1[idx]))
            return repl
        
        #def repl(obj1, )

        def create_eqs():
            # eq1
            eq1 = manim.MathTex(r'X0 \times f0 + X1 \times f1 + X2 \times f2 = Y', color=font_color,
                                 substrings_to_isolate=all_identifiers)  
            eq1_by_index = {k:eq1.index_of_part_by_tex(k) for k in ['X0', 'X1', 'X2', 'f0', 'f1', 'f2', 'Y']}
            eq1_by_obj = {k:eq1.get_part_by_tex(k) for k in ['X0', 'X1', 'X2', 'f0', 'f1', 'f2', 'Y']}
            x0, x1, x2 = (create_2d_vec(num_input_boxes, manim.Y_AXIS, colors[i]) for i in range(len(mix)))
            f0, f1, f2 = (manim.Integer(i, color=font_color) for i in mix)
            #f0, f1, f2 = (i for i in mix)
            y = create_2d_vec(num_input_boxes,manim.Y_AXIS, result_color)
            #xx = manim.MobjectMatrix([[x0.copy(), x1.copy(), x2.copy()]])
            xx = manim.Matrix([['X0', 'X1', 'X2']])
            ff = manim.IntegerMatrix([[0], [1], [2]])
            print(manim.matrix_to_tex_string([[0,1, 2]]))
            #yy = manim.MobjectMatrix(np.array([[y.copy()]]))  
            xx.get_mob_matrix()[0,0].become(x0.copy().move_to(xx.get_mob_matrix()[0, 0]))
            xx.get_mob_matrix()[0,1].become(x1.copy().move_to(xx.get_mob_matrix()[0, 1]))
            xx.get_mob_matrix()[0,2].become(x2.copy().move_to(xx.get_mob_matrix()[0, 2]))
            
            repl(eq1, 'X0', x0)
            repl(eq1, 'X1', x1)
            repl(eq1, 'X2', x2)
            repl(eq1, 'f0', f0)
            repl(eq1, 'f1', f1)
            repl(eq1, 'f2', f2)
            repl(eq1, 'Y', y)
            #eq1_by_obj['X1'].become(x1)
            #eq1_by_obj['X2'].become(x2)
            #eq1_by_obj['f0'].become(f0)
            #eq1_by_obj['f1'].become(f1)
            #eq1_by_obj['f2'].become(f2)
            #eq1_by_obj['Y'].become(y)
            # eq2

            replacements = []
            #replacements = [(x0, m[0,0]), (x1, m[0, 1]), (x2, m[0, 1]),
            #                (f0, ff[0, 0]), (f1, m[1, 0]), (f2, m[2, 0]), (y, yy[0, 0])]
            eq2 = manim.Tex('XX ff = YY', color=font_color, substrings_to_isolate=['XX', 'YY', 'ff'])
            eq2.get_part_by_tex('XX').become(xx)
            #eq2.get_part_by_tex('ff').become(ff)
            #eq2.get_part_by_tex('yy').become(yy)
            return eq1, eq2, replacements
        eq1, eq2, replacements = create_eqs()
        #eq2.shift([0, -2, 0])
        self.add(eq2)
        #self.add(eq1)
        #self.wait(1)
        #self.play(*[manim.ReplacementTransform(eq1[r[0]], eq2[r[1]]) for r in replacements])
        #self.play(manim.ReplacementTransform(eq1, eq2))
        #self.play(eq1.animate.fade(darkness=1), rate=4.0)

class CombiningColumns2(manim.Scene):
    def construct(self):
        vecs = [create_2d_vec(num_input_boxes, manim.Y_AXIS, colors[i]) for i in range(len(mix))]
        vecs.append(create_2d_vec(num_input_boxes,manim.Y_AXIS, result_color))
        #          0     
        #         X0 xf0 + X1 xf1 X2 xf2 = Y
        #         [X0 & X1 & X2][f1 \\ f2 \\ f3] = [Y]
        vec_identifiers = ['X0', 'X1', 'X2', 'Y']
        factor_identifiers = ['f0', 'f1', 'f2']
        all_identifiers = vec_identifiers + factor_identifiers + ['=']
        eq1 = manim.MathTex('X0',       #0
                            '\\times',  #1
                            'f0',       #2
                            '+',        #3
                            'X1',       #4
                            '\\times',  #5
                            'f1',       #6
                            '+',        #7
                            'X2',       #8
                            '\\times',  #9
                            'f2',       #10
                            '=',        #11
                            'Y',        #12
                            color=font_color, substrings_to_isolate=all_identifiers)
        eq2 = manim.MathTex('\\left[',  #0
                            'X0',       #1
                            'X1',       #2
                            'X2',       #3
                            '\\right]', #4
                            '=',        #5
                            '\\left[',  #6
                            'Y',        #7
                            '\\right]', #8
                            color=font_color,
                           substrings_to_isolate=vec_identifiers) 
        #changes = [(0, 2, 4, 6, 8, 10, 12),
        #           (1, , 3,)]        
        eq1 = manim.MathTex(r'X0 \times f0 + X1 \times f1 + X2 \times f2 = Y', color=font_color,
                           substrings_to_isolate=all_identifiers)
        eq2 = manim.MathTex(
                        r'\left[ X0 X1 X2 \right] \big[ f0  f1 f2 \big] = \left[ Y \right]',
                        #'\\left[ \\begin{array}{ccc}X0 & X1 & X2\\end{array} \\right] f0 f1 f0 Y',
                        #r'\left[ X0  X1  X2  \right] \big[ f0  f1 f2 \big] = \left[ Y \right]',
                        #r'\left[ X0 X1 X2 \right] \begin{bmatrix} f0 \\ f1 \\ f2 \end{bmatrix} = \left[ Y \right]',
                        color=font_color, substrings_to_isolate=all_identifiers)
        eq2.shift([0, -2, 0])
        replacements = []
        # Insert column vectors
        for i in range(len(vecs)):
            idx1 = eq1.index_of_part_by_tex(vec_identifiers[i])
            idx2 = eq2.index_of_part_by_tex(vec_identifiers[i])
            replacements.append((idx1, idx2))
            cp1 = vecs[i].copy()
            cp2 = vecs[i].copy()
            cp1.move_to(eq1[idx1])
            cp2.move_to(eq2[idx2])
            eq1[idx1].become(cp1)
            eq2[idx2].become(cp2) 
        for f in factor_identifiers + ['=']:
            idx1 = eq1.index_of_part_by_tex(f)
            idx2 = eq2.index_of_part_by_tex(f)
            replacements.append((idx1, idx2)) 
        #self.add(eq2)
        self.add(eq1)
        #self.play(manim.Write(eq1))
        self.wait(1)
        eq2cp = eq2.copy()
        self.play(*[manim.ReplacementTransform(eq1[r[0]], eq2[r[1]]) for r in replacements])
        self.add(eq2cp)
        self.wait(1)
        #self.play(manim.ReplacementTransform(eq1, eq2))
        #self.play(eq1.animate.fade(darkness=1), rate=4.0)
        #self.add(eq2)

In [15]:
# Testing out MobjectMatrix.
%%manim -ql Test

class Test(manim.Scene):
    def construct(self):
        def f1(m):
            print(m)
            return manim.Circle(m)
        def f2(m):
            print(m)
            return m
        circles = [[manim.Circle(1), manim.Circle(2), manim.Circle(3), manim.Circle(4)]]     
        circles_np = np.array(circles, dtype=object)
        ints = [[1, 2, 3, 4]]
        m1 = manim.MobjectMatrix(ints,    element_to_mobject=f1)
        # m2 = manim.MobjectMatrix(circles, element_to_mobject=f2)
        c = circles_np[0,0]

UsageError: Line magic function `%%manim` not found.
